In [90]:
# salesforce fixup

import io
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict
from simple_salesforce import Salesforce                   # !pip3 install salesforce-reporting
from pardot.client import APIClient                        # !pip3 install pardot-api-client
import pytz
import datetime
from uszipcode import SearchEngine                         # pip3 install uszipcode
search = SearchEngine(simple_zipcode=False)                # set simple_zipcode=False to use rich info database
# !pip3 install --upgrade uszipcode                          # upgrade databases
from simple_salesforce import Salesforce                   # !pip3 install simple-salesforce
from salesforce_reporting import Connection, ReportParser  
with open('.sfdc') as f:
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=uname, password=spass, security_token=stoken)          # authenticate
end = datetime.datetime.now(pytz.UTC)                                           # salesforce API requires UTC
print("@ ", end)
search = SearchEngine(simple_zipcode=True)                # False uses rich info database

@  2019-08-28 14:30:48.099092+00:00


In [91]:
# get territories

u_soql = (
"SELECT Id, Name, Role_Text__c, Territory_BDR_Id__c, Territory_BDR_Name__c, Territory_Name__c, Territory__c "
"FROM User "
"WHERE Territory_BDR_Id__c != NULL AND Territory__c != NULL"
)
# u_soql
u_query = sf.query(u_soql)
# u_query
records = [dict(
                  Id = rec['Id'],
                 Rep = rec['Name'],
                Role = rec['Role_Text__c'],
               BDRId = rec['Territory_BDR_Id__c'],
                 BDR = rec['Territory_BDR_Name__c'],
           Territory = rec['Territory_Name__c'],
           Geography = rec['Territory__c'])
    for rec in u_query['records']]
u_qdf = pd.DataFrame(records)
u_qdf

,BDR,BDRId,Geography,Id,Rep,Role,Territory
0,Mario Pineda,0052H00000Byt5y,CT;DE;ME;MA;NH;NJ;NY;PA;RI;VT,0053400000CK2hLAAT,Mike Moloughney,Corporate Enterprise Team,CESE_NE
1,John Nitchen,0053400000Bohec,AZ;CA;KS;MO;NV;NM;OK;TX,0053400000AEDLKAA5,John Guilford,Corporate Enterprise Team,CESE_SW
2,Adrian Grodzki,0052H00000Byg1o,CO;ID;IL;IN;IA;MI;MN;NE;ND;OH;OR;SD;UT;WA;WI;WY,00534000009vDsaAAE,Tuan Nguyen,Corporate Enterprise Team,CESE_NW
3,Adrian Grodzki,0052H00000Byg1o,AL;AR;DC;FL;GA;KY;LA;MD;MS;NC;SC;TN;VA;WV,00534000009twvaAAA,Joe Heddy,Corporate Enterprise Team,CESE_SE
4,John Nitchen,0053400000Bohec,CT;ME;ME;MD;MA;NH;NJ;NY;PA;RI;VT;WV,00580000007LGMXAA4,Greg Prete,Corporate Enterprise 2 Team,CMSE_NE
5,Derek Bell,0052H00000BynhN,AL;AR;FL;GA;KS;KY;LA;MS;MO;NC;SC;TN;VA,0053400000CHTAvAAP,Trevor Fair,Corporate Enterprise 2 Team,CMSE_SE_GAM
6,Mario Pineda,0052H00000Byt5y,AK;AZ;CA;HI;NV;NM;OK;TX;UT,0053400000AEDKqAAP,Alexandra Singer,Corporate Enterprise 2 Team,CESE_SW
7,Christine Kendall,0053400000CQm13,AL;AR;CT;DC;DE;FL;GA;IL;IN;IA;KY;LA;ME;MD;MI;M...,00580000003ulxYAAQ,Tim Detota,Guest Experience Team,GXSE_EA_GAM
8,Christine Kendall,0053400000CQm13,AL;AZ;CA;CO;HI;ID;KS;MT;NE;NV;NM;OK;OR;TX;UT;W...,00580000003Rz0IAAS,Mark Filler,Guest Experience Team,GXSE_WE_GAM
9,Christine Kendall,0053400000CQm13,AL;CT;DC;DE;FL;GA;IN;KY;LA;ME;MD;MA;MS;NH;NJ;N...,00580000001f9aEAAQ,Andrew Daughters,Guest Experience Team,GXSE_EA_HOS


In [88]:
# get records

l_soql = (
"SELECT Id, OwnerID, Account_Development_rep__c, FWILeadOwner__c, Role__c, LeadSource, Lead_Type__c, "
       "utm_content_last__c, Industry_Vertical__c, "
       "StateCode, PostalCode, NumberOfEmployees "
"FROM Lead "
"WHERE LeadSource IN ('Madison', 'Integrate', 'Undertone') "
    "AND (Lead_Type__c = 'None' OR FWILeadOwner__c = 'SOS') "
    
    
    SELECT Id, OwnerID, Account_Development_rep__c, Territory_BDR__c, FWILeadOwner__c, Role__c, LeadSource, Lead_Type__c, 
       Industry_Vertical__c, StateCode
FROM Lead 
WHERE LeadSource IN ('Madison', 'Integrate', 'Undertone')
    
# "LIMIT 10"
)
l_soql
l_query = sf.query(l_soql)
l_query
records = [dict(
                  Id = rec['Id'],
             OwnerID = rec['OwnerId'],
               Owner = rec['FWILeadOwner__c'],
                Role = rec['Role__c'],
              Source = rec['LeadSource'],
         Disposition = rec['Lead_Type__c'],
             Content = rec['utm_content_last__c'],
            Vertical = rec['Industry_Vertical__c'],
               State = rec['StateCode'],
                 Zip = rec['PostalCode'],
           Employees = rec['NumberOfEmployees'])
    for rec in l_query['records']]
l_qdf = pd.DataFrame(records)
l_qdf

""


In [84]:
# l_qdf['OwnerID'] = np.where(l_qdf['State']=u_qdf['Geography'], 'yes', 'no')
# lambda function to check if they match. row.name gets the index
l_qdf['NewOwnerID'] = l_qdf.apply(lambda row: 1 if row[0] == u_qdf['Geography'][row.name] else 0, axis = 1)
u_qdf.str.contains('og', regex=False)

# Solution 1 (checking directly if 'Invoice' string is in the 'Ref' string)
df4 = df2.copy()
df4['Invoice'] = [val for idx, val in enumerate(df1['Invoice']) if val in df2['Ref'][idx]]
df_m4 = df1.merge(df4[['Amount', 'Invoice']], on='Invoice')

""


In [98]:
df1 = pd.DataFrame({'Invoice': ['20561', '20562', '20563', '20564'],
                    'Currency': ['EUR', 'EUR', 'EUR', 'USD']})
df2 = pd.DataFrame({'Ref': ['20561', 'INV20562', 'INV20563BG', '20564'],
                    'Type': ['01', '03', '04', '02'],
                    'Amount': ['150', '175', '160', '180'],
                    'Comment': ['bla', 'bla', 'bla', 'bla']})
df1

,Invoice,Currency
0,20561,EUR
1,20562,EUR
2,20563,EUR
3,20564,USD


In [97]:
df2

,Ref,Type,Amount,Comment
0,20561,01,150,bla
1,INV20562,03,175,bla
2,INV20563BG,04,160,bla
3,20564,02,180,bla


In [99]:
df4 = df1.copy()
def get_amount(x):
    return df2.loc[df2['Ref'].str.contains(x), 'Amount'].iloc[0]
df4['Amount'] = df4['Invoice'].apply(get_amount)

df4

,Invoice,Currency,Amount
0,20561,EUR,150
1,20562,EUR,175
2,20563,EUR,160
3,20564,USD,180


In [101]:
# Solution 1 (checking directly if 'Invoice' string is in the 'Ref' string)
df4 = df2.copy()
df4['Invoice'] = [val for idx, val in enumerate(df1['Invoice']) if val in df2['Ref'][idx]]
df_m4 = df1.merge(df4[['Amount', 'Invoice']], on='Invoice')

# Solution 2 (regex)
import re
df5 = df2.copy()
df5['Invoice'] = [re.findall(r'(\d{5})', s)[0] for s in df2['Ref']]
df_m5 = df1.merge(df5[['Amount', 'Invoice']], on='Invoice')
df_m4

,Invoice,Currency,Amount
0,20561,EUR,150
1,20562,EUR,175
2,20563,EUR,160
3,20564,USD,180


In [45]:
# df = pd.read_csv('data/15Pct.csv')
# with open('csvfile.csv', 'w') as csvfile: f = csv.writer(csvfile) f.writerows(items)  # write to csv file
# qdf['PostalCode'] = qdf['PostalCode'].str[:5]       # trim off zip+4
# def zco(x):
#    return search.by_zipcode(x).state
# qdf['StateCode'] = qdf['PostalCode'].fillna(0).astype(int).apply(zco)
# qdf.to_csv (r'madison.csv', index = None, header=True
# qdfj = qdf.to_json(orient='records')
# qdfj
# qdf
# sf.bulk.Contact.insert(data)
# sf.bulk.Contact.upsert(data, 'Id')
# sf.bulk.Lead.update(qdfj)
# zipcode = search.by_zipcode("06916")
# zipcode.to_json()                                       # to json
# zipcode.to_dict()                                       # to dict
# zip = zipcode.values()                                  # to list
# zipcode

In [ ]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}